In [ ]:
import cv2
import time

import sys
sys.path.insert(0, '..')

from game.game import ProductOwnerGame
from environment.backlog_env import BacklogEnv
from environment.environment import ProductOwnerEnv
from environment.reward_sytem.base_reward_system import BaseRewardSystem
from environment.userstory_env import UserstoryEnv

from pipeline.study_agent import load_dqn_agent

from web_interaction import GameImageParser, GameCoordinator, WebController

from pet_logging import get_logger

from logging import Logger

In [ ]:
def log_game_state(game: ProductOwnerGame, logger: Logger):
    logger.info(f'Sprint {game.context.current_sprint}')
    logger.info(f'Money {game.context.get_money()}')
    logger.info(f'Loyalty {game.context.get_loyalty()}')
    logger.info(f'Customers {game.context.customers}')
    logger.info(f'Credit {game.context.credit}')

In [ ]:
logger = get_logger('WebInteraction')

image_parser = GameImageParser('../web_interaction/templates')
game_coordinator = GameCoordinator(image_parser)
game = ProductOwnerGame()

userstory_env = UserstoryEnv(4, 0, 0)
backlog_env = BacklogEnv(12, 0, 0, 0, 0, 0)
reward_system = BaseRewardSystem(config={})
env = ProductOwnerEnv(userstory_env, backlog_env, with_info=True, reward_system=reward_system)
env.game = game

web_controller = WebController(game, game_coordinator, logger)

In [ ]:
agent = load_dqn_agent('../models/credit_start_model.pt')

In [ ]:
driver = web_controller.open_game()
iframe = web_controller.find_game(driver)
web_controller.open_full_sceen(driver)
web_controller.wait_loading(iframe)
web_controller.start_game(driver, iframe)
image = web_controller.take_screenshot(iframe)

In [ ]:
game_coordinator.skip_tutorial(game)
game_coordinator.insert_user_stories_from_image(game, image)

game.userstories.stories_list

In [ ]:
game_coordinator.update_header_info(game, image)

log_game_state(game, logger)

In [ ]:
while not game.context.done:
    state = env.recalculate_state()
    # print(state)

    info = env.get_info()
    # print(info)

    action = agent.get_action(state, info)
    logger.info(f'Action id: {action}')

    time.sleep(1)
    web_controller.apply_web_action(action, driver, iframe, env)

    if env.game.context.current_sprint >= 35:
        logger.warning("Reached credit end!")
        break

In [ ]:
raise Exception('Break before empty sprinst')

In [ ]:
log_game_state(game, logger)

In [ ]:
while not game.context.done:
    time.sleep(1)
    web_controller.apply_web_action(0, driver, iframe, env)

In [ ]:
iframe.rect

In [ ]:
raise Exception('Break before web driver exit')

In [ ]:
driver.quit()